In [ ]:
# !pip install keras 
# !pip install scikit-learn
# !pip install matplotlib
# !pip install tensorflow
# !pip install numpy<2
# !pip install pyspark

In [4]:
month = "June"
file_path = "data/filtered_pred_"+month+"2024.csv"

In [5]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
# from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional


In [6]:
# Import necessary libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum, col, hour, concat_ws, to_date, date_format
# Stop any existing Spark session
# Step 1: Initialize a Spark session
spark = SparkSession.builder \
    .appName("BigDataProcessing") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()
# Step 2: Load your CSV file into a Spark DataFrame
data = spark.read.csv(file_path, header=True, inferSchema=True)


In [7]:

data = data.groupBy("DATE_HOUR") \
    .agg(
        spark_sum("NO_OF_ADULT").alias("NO_OF_ADULT"),
        spark_sum("NO_OF_CHILD").alias("NO_OF_CHILD")
    )
df = data.toPandas()

In [8]:
import pandas as pd
# Step 7: Convert 'DATE_HOUR' to datetime format, then set as the index
df['DATE_HOUR'] = pd.to_datetime(df['DATE_HOUR'], format='%Y-%m-%d %H')
df.set_index('DATE_HOUR', inplace=True)
df.sort_index(inplace=True)



In [9]:
# Step 7: Scale the data

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

scaled_data = scaler.fit_transform(df)


# print(scaled_data)


In [10]:
spark.stop()

In [11]:

# Create Train and Test datasets
Train = scaled_data
# Save the Test dataset



In [12]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
# Step 10: Create TimeseriesGenerator for training data
n_input = 24  # Use the last 24 hours for prediction
n_features = 2  # Number of features (NO_OF_ADULT and NO_OF_CHILD)

generator = TimeseriesGenerator(Train, Train, length=n_input, batch_size=1)


In [13]:
# Define the LSTM model
model = Sequential([
    Bidirectional(LSTM(units=100, activation='tanh', return_sequences=True), input_shape=(n_input, n_features)),
    Bidirectional(LSTM(units=50, activation='tanh')),
    Dense(units=n_features, activation='linear')  # 'linear' is appropriate for regression
])

2025-01-10 14:31:02.620944: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/jeev/.local/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# compile the model

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

In [15]:
from keras.callbacks import EarlyStopping

# model training
# Train the model
epochs = 40  # More epochs can lead to better performance

# Add EarlyStopping callback to stop training if the validation loss does not improve
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Train the model with the EarlyStopping callback
history = model.fit(generator, epochs=epochs, verbose=1, callbacks=[early_stopping])


Epoch 1/40


/home/jeev/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


597/597 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.0292 - mae: 0.1245
Epoch 2/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 0.0107 - mae: 0.0735
Epoch 3/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.0073 - mae: 0.0564
Epoch 4/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 0.0066 - mae: 0.0576
Epoch 5/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 0.0053 - mae: 0.0522
Epoch 6/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - loss: 0.0052 - mae: 0.0512
Epoch 7/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - loss: 0.0051 - mae: 0.0498
Epoch 8/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 0.0047 - mae: 0.0459
Epoch 9/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 0.0043 - mae: 0.0451
Epoch 10/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - loss: 0.0035 - mae: 0.0398
Epoch 11/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 0.0039 - mae: 0.0432
Epoch 12/40
597/597 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - loss: 0.0037 - mae: 0.0416
Epoch 13/40
59

In [16]:
model.save(month+'.keras')